In [27]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# API key
from api_keys import g_key

In [28]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [29]:
# Read in csv
weather_df = pd.read_csv("weatherPyData.csv")
weather_df.head()

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,longyearbyen,78.22,15.64,24.80,62,1008,5.82,20
1,kapaa,22.08,-159.32,73.96,73,1021,20.80,90
2,fortuna,40.60,-124.16,41.18,100,1023,10.29,1
3,severo-kurilsk,50.68,156.12,36.86,80,1011,16.96,100
4,cayenne,4.93,-52.33,77.00,100,1012,13.65,41


In [30]:
# Store latitude and longitude in locations
locations = weather_df[["latitude", "longitude"]]

# Fill NaN values and convert to float
humid = weather_df["humidity"]

In [31]:
# Plot Heatmap, create layers and display
fig = gmaps.figure(map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(locations, weights=humid, dissipating=True)



fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# Make masks to narrow conditions to ideal weather 
mask1 = (weather_df.temperature >= 70) & (weather_df.temperature < 80)
mask2 = weather_df.wind_speed < 10
mask3 = weather_df.cloudiness == 0
mask4 = mask1 & mask2 & mask3

ideal_spots = weather_df.loc[mask4].reset_index(drop=True)
ideal_spots

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,vila velha,-20.33,-40.29,77.23,54,1019,9.17,0
1,cidreira,-30.18,-50.21,70.29,57,1016,5.70,0
2,hermanus,-34.42,19.23,71.01,58,1021,3.00,0
3,ihosy,-22.40,46.12,73.11,44,1017,5.46,0
4,buritis,-15.62,-46.42,77.81,46,1016,4.88,0
5,ilhabela,-23.78,-45.36,73.54,54,1018,2.37,0
6,carnarvon,-24.87,113.63,73.40,100,1016,8.05,0
7,sao sebastiao,-23.76,-45.41,74.71,52,1018,2.44,0
8,brive-la-gaillarde,45.25,1.50,79.32,56,1013,1.99,0
9,springbok,-29.66,17.89,75.99,13,1018,7.92,0


In [44]:
# Check if I can view new head

ideal_spots.head(9)



,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,vila velha,-20.33,-40.29,77.23,54,1019,9.17,0
1,cidreira,-30.18,-50.21,70.29,57,1016,5.70,0
2,hermanus,-34.42,19.23,71.01,58,1021,3.00,0
3,ihosy,-22.40,46.12,73.11,44,1017,5.46,0
4,buritis,-15.62,-46.42,77.81,46,1016,4.88,0
5,ilhabela,-23.78,-45.36,73.54,54,1018,2.37,0
6,carnarvon,-24.87,113.63,73.40,100,1016,8.05,0
7,sao sebastiao,-23.76,-45.41,74.71,52,1018,2.44,0
8,brive-la-gaillarde,45.25,1.50,79.32,56,1013,1.99,0


In [34]:
# Check URL and see if it returns back a country
# Repeat in cell below 

params = {"key": g_key}

base_url = "https://maps.googleapis.com/maps/api/geocode/json"

lat = ideal_spots["latitude"][1]
lng = ideal_spots["longitude"][1]

# update address key value
params['latlng'] = f"{lat},{lng}"

# make request
thing = requests.get(base_url, params=params)

thing = thing.json()
thing["results"][0]["address_components"]


for addressComp in thing["results"][0]["address_components"]:
    if addressComp["types"][0] == "country":
        print(addressComp["long_name"])

Brazil


In [45]:
def country_city(lat, long):    
    rtnCountry = ""
    
    params = {"key": g_key}

    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    lat = ideal_cities["latitude"][indx]
    lng = ideal_cities["longitude"][indx]

    # update address key value
    params['latlng'] = f"{lat},{lng}"

    # make request
    thing = requests.get(base_url, params=params)

    thing = thing.json()
    thing["results"][0]["address_components"]


    for addressComp in thing["results"][0]["address_components"]:
        if addressComp["types"][0] == "country":
            rtnCountry = addressComp["long_name"]
            
    return rtnCountry

In [46]:
countries = []
for indx, row in ideal_spots.iterrows():
    countries.append(getCountryForCity(row.latitude, row.longitude))

In [48]:
#Check and see if countries were appended 
countries

['Brazil',
 'Brazil',
 'South Africa',
 'Madagascar',
 'Brazil',
 'Brazil',
 'Australia',
 'Brazil',
 'France',
 'South Africa',
 'China',
 'Brazil',
 'France']

In [49]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

lat = ideal_cities["latitude"][0]
lng = ideal_cities["longitude"][0]

params["location"] = f"{lat},{lng}"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

name_address = requests.get(base_url, params=params)
name_address = name_address.json()

name_address["results"][5]["name"]

'Quality Suites Vila Velha'

In [39]:


names = []
addresses = []
ratings = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in ideal_cities.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    
    try:
        names.append(name_address["results"][0]["name"])
        addresses.append(name_address["results"][0]["vicinity"])
        ratings.append(name_address["results"][0]["rating"])
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        names.append("")
        addresses.append("")
        ratings.append("")

In [40]:
names

['Golden Tulip Porto Vitória',
 'Pond Country Club',
 'The Marine Hermanus',
 'Tamana Hôtel',
 'Hotel Rainha do Vale',
 'Ilha Flat Hotel',
 'Coral Coast Tourist Park',
 'Hotel Arrastão',
 'Auberge du Cheval Blanc',
 'Springbok Inn by Country Hotels',
 'New City International Hotel',
 'Hotel Guarapousada',
 "Maison Vin d'Hus"]

In [41]:
ideal_cities["country"] = countries
ideal_cities["hotels"] = names
ideal_cities["addresses"] = addresses
ideal_cities["ratings"] = ratings

ideal_cities

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness,country,hotels,addresses,ratings
0,vila velha,-20.33,-40.29,77.23,54,1019,9.17,0,Brazil,Golden Tulip Porto Vitória,"Av. Nossa Sra. dos Navegantes, 635 - Enseada d...",4.6
1,cidreira,-30.18,-50.21,70.29,57,1016,5.70,0,Brazil,Pond Country Club,"Estrada RS 784 KM 7 - Lagoa, Cidreira",4.6
2,hermanus,-34.42,19.23,71.01,58,1021,3.00,0,South Africa,The Marine Hermanus,"Main Road, Hermanus",4.5
3,ihosy,-22.40,46.12,73.11,44,1017,5.46,0,Madagascar,Tamana Hôtel,Ihosy,2.9
4,buritis,-15.62,-46.42,77.81,46,1016,4.88,0,Brazil,Hotel Rainha do Vale,"ua Centro, R. Floresta, 767 - Centro, Buritis",3.5
5,ilhabela,-23.78,-45.36,73.54,54,1018,2.37,0,Brazil,Ilha Flat Hotel,"Av. Princesa Isabel, 747 - Praia do Perequê, I...",4.5
6,carnarvon,-24.87,113.63,73.40,100,1016,8.05,0,Australia,Coral Coast Tourist Park,"108 Robinson St, Carnarvon",4.3
7,sao sebastiao,-23.76,-45.41,74.71,52,1018,2.44,0,Brazil,Hotel Arrastão,"Av. Dr. Manoel Hipólito do Rêgo, 2097 - Arrast...",4.3
8,brive-la-gaillarde,45.25,1.50,79.32,56,1013,1.99,0,France,Auberge du Cheval Blanc,"15 Rue Jean Cariven, Allassac",4.6
9,springbok,-29.66,17.89,75.99,13,1018,7.92,0,South Africa,Springbok Inn by Country Hotels,"Voortrekker St, Springbok",4.0


In [42]:
spot_info = []

for indx, row in ideal_cities.iterrows():
    info_box = f"""
                <dl>
                <dt>Hotel Name</dt><dd>{row.hotels}</dd>
                <dt>City</dt><dd>{row.cities}</dd>
                <dt>Country</dt><dd>{row.country}</dd>
                </dl>
                """
    
spot_info.append(info_box)

In [50]:
# Plot heatmap, create layers, add markers,and then display
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)


map_layer = gmaps.heatmap_layer(locations, weights=humid, dissipating=True)



fig.add_layer(heat_layer)


coordinates = ideal_spots[["latitude", "longitude"]]
marker_layer = gmaps.marker_layer(coordinates, info_box_content=city_info)
fig.add_layer(marker_layer)


fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…